In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import torch

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
df_train.head()

In [ ]:
X=np.array(df_train.drop(['label'],axis=1),dtype=float)
Y=np.array(df_train['label'],dtype=float)

In [ ]:
for i in range(3):
    fig=plt.figure()
    plt.imshow(X[i].reshape([28,28]))
    plt.title(str(Y[i]))

In [ ]:
X_train = torch.tensor(X_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
Y_train = torch.tensor(Y_train,dtype=torch.float32)
Y_test = torch.tensor(Y_test,dtype=torch.float32)

In [ ]:
batch_size = 500
max_iter = 50000
lr = 1e-3
i = 0

model = torch.nn.Linear(784, 10)
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=lr)

history = []

while 1:
    i += 1
    ind = np.random.randint(0,len(X_train),batch_size)
    X_batch = torch.tensor(X_train[ind])
    Y_batch = torch.tensor(Y_train[ind])

    
    Y_pred = model(X_batch)[:,:]
    loss = torch.mean(criterion(Y_pred,Y_batch.type(torch.LongTensor)))
    
    loss.backward()
    optim.step()
    optim.zero_grad()
    
    history.append(loss.data.numpy())
    if i % 1000 == 0:
        print(f'Iteration = {i} Loss = {loss.data.numpy()}')
    
    if loss.data.numpy() < 0.5:
        break

In [ ]:
Y_pred = model(X_test)
_, predicted = torch.max(Y_pred.data, 1)

correct = 0
total = len(predicted)

for i in range(len(predicted)):
    if predicted[i] == Y_test[i]:
        correct += 1
        
accuracy = 100 * correct / total
print(f'Accuracy = {accuracy}')

In [ ]:
from sklearn import metrics

# cm = metrics.confusion_matrix(Y_test, predictions, labels=clf.classes_)
disp = metrics.ConfusionMatrixDisplay(metrics.confusion_matrix(Y_test.detach().numpy(), predicted.detach().numpy()))
disp.plot()

In [ ]:
X=torch.tensor(np.array(df_test,dtype=float),dtype=torch.float32)

Y_pred = model(X)
_, predicted = torch.max(Y_pred.data, 1)

df_test.index -= 1
ImageId = df_test.index

output = pd.DataFrame({'ImageID': ImageId, 'Label': predicted})
output.to_csv('submission.csv', index=False)